In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# For LangSmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [11]:
# created langchain environment in conda and selected that as kernel to run the code
from langchain_openai import ChatOpenAI

# no need to pass api key as we already loaded it from .env file
llm=ChatOpenAI(model='gpt-3.5-turbo')
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x00000223097C8A90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000223097E05B0> root_client=<openai.OpenAI object at 0x000002230971C5E0> root_async_client=<openai.AsyncOpenAI object at 0x00000223097C8CD0> model_kwargs={} openai_api_key=SecretStr('**********')


In [ ]:
# passing input and getting response from the llm
result=llm.invoke("explain gen ai")
print(result)

#### PromptTemplate
PromptTemplate is something using which we can instruct our llm model how to behave. PromptTemplate take as input a dictionary, where each key represents a variable in the prompt template to fill in. 
- ChatPromptTemplate

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answer based on the question"),
    ("user","{input}")
])
print(prompt)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


#### Chaining using | operator
this | is saying that we need to combine all things connected with | and use it in passing to llm

In [ ]:
chain=prompt|llm
response=chain.invoke({"input":"can you tell me about langsmith?"})
print(response)

#### Output Parser
It is responsible for providing the response how we want to display it. 
We can also create our own custom output parsers.
- StrOutputParser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()

chain=prompt|llm|output_parser
response=chain.invoke({"input":"can you tell me about langsmith?"})
print(response)